In [69]:
import numpy as np
from scipy.optimize import linprog

def generate_bids(n, m, p_bar):
    """
    Generate a sequence of random bids.
    :param n: Total number of bids.
    :param m: Number of items.
    :param p_bar: Ground truth price vector.
    :return: Array of bids.
    """
    bids = []
    for _ in range(n):
        a_k = np.random.choice([0, 1], size=m)  # Generate a_k
        pi_k = np.dot(p_bar, a_k) + np.random.normal(0, np.sqrt(0.2))  # Calculate bid price
        bids.append((a_k, pi_k))
    return bids

def solve_offline_lp(bids, m, b_i):
    c = -np.array([pi_k for _, pi_k in bids])  # Negative for maximization
    A = np.array([a_k for a_k, _ in bids]).T  # Transpose to match dimensions
    b = b_i * np.ones(m)

    # Solving the LP
    result = linprog(c, A_ub=A, b_ub=b, bounds=(0, 1), method='highs')

    if result.success:
        return -result.fun  # Revenue (negate because of maximization)
    else:
        raise ValueError("Offline LP did not converge")

def solve_partial_lp_dual(bids, k, n, b_i):
    # Objective function: maximize sum(pi_j * x_j) for j=1 to k
    c = -np.array([pi_k for _, pi_k in bids[:k]])

    # Constraints: sum(a_ij * x_j) <= (k/n) * b_i for all i
    A = np.array([a_k for a_k, _ in bids[:k]]).T
    b = (k / n) * np.array(b_i)

    # Bounds for decision variables: 0 <= x_j <= 1
    x_bounds = [(0, 1) for _ in range(k)]

    # Solve the linear program
    result = linprog(c, A_ub=A, b_ub=b, bounds=x_bounds, method='highs')

    # print(result.ineqlin.get('marginals'))

    if result.success:
        # The dual variable corresponding to the inequality constraints A_ub * x <= b_ub
        return result.get('slack'), -1*result.ineqlin.get('marginals'), result.get('fun')
    else:
        raise ValueError("failed to find a solution")

In [70]:
diff_dplm = []
diff_ahdla = []
revenues = {}

n = 10000  # Total number of bids
m = 10     # Number of items
b_i = np.ones(m) * 1000  # Bid cap for all i
# Fixed ground truth price vector (p_bar) - set to ones for simplicity
p_bar = np.ones(m)  # Vector of ones
k_values = [50, 100, 200]  # Different k values to test
# Regenerate bids with the fixed p_bar
bids_fixed = generate_bids(n, m, p_bar)

# offline
offline_revenue_value = solve_offline_lp(bids_fixed, m, b_i)
print(f"Offline revenue: {offline_revenue_value}")

Offline revenue: 11295.824597861341


In [71]:
# Problem 1
def run_slpm_static(bids, k, n, b_i):
    revenue = 0
    remaining_capacity = np.array(b_i)
    # Solve the partial LP for the first k bids to get dual prices
    slack, dual_price, k_revenue = solve_partial_lp_dual(bids, k, n, b_i)
    # revenue += -k_revenue
    # remaining_capacity = np.array(b_i-((k/n)*b_i-slack))

    for i, (a_k, pi_k) in enumerate(bids):
        if i >= k:
            # Allocate based on the decision rule using y_bar
            if pi_k > np.dot(a_k, dual_price) and all(remaining_capacity - a_k >= 0):
                revenue += pi_k
                remaining_capacity -= a_k

    return revenue

for k in k_values:
    revenue = run_slpm_static(bids_fixed, k, n, b_i)
    revenues[k] = revenue

for slpm_revenue, k in zip(revenues.values(), k_values):
    print(f"SLPM Static revenue: {slpm_revenue} at k={k}")

SLPM Static revenue: 8704.104241903948 at k=50
SLPM Static revenue: 9034.464855685475 at k=100
SLPM Static revenue: 10359.845241415733 at k=200


The increase in revenue as k grows suggests that having more bidder information upfront (i.e., a larger k) allows the algorithm to make more informed decisions which result in higher revenue.

It may have two tradeoffs here, one is tradeoff between cost of computation and accuracy, since with a large k we will need more compuration resources. Another tradeoff is that a larger k implies a delay in decision-making since more bids must be collected before making allocations, which could be a trade-off in real-time scenarios.


In [72]:
# Problem 2
def run_slpm_dynamic(bids, k, n, b_i, offline_revenue_value):
    global diff_dplm
    revenue = 0
    remaining_capacity = np.array(b_i)

    # Solve the partial LP for the first k bids to get dual prices
    slack, dual_price, k_revenue = solve_partial_lp_dual(bids, k, n, b_i)
    # revenue += -k_revenue
    # remaining_capacity = np.array(b_i-((k/n)*b_i-slack))

    target = 2*k

    for i, (a_k, pi_k) in enumerate(bids):
        if i >= k:
            if i == target:
                slack, dual_price, _ = solve_partial_lp_dual(bids, target, n, b_i)
                print(dual_price)
                target *= 2
            # Allocate based on the decision rule using y_bar
            if pi_k > np.dot(a_k, dual_price) and all(remaining_capacity - a_k >= 0):
                revenue += pi_k
                remaining_capacity -= a_k
            diff_dplm.append(revenue-((i+1)/n)*offline_revenue_value)


    return revenue

revenue = run_slpm_dynamic(bids_fixed, 50, n, b_i, offline_revenue_value)
print(f"SLPM Dynamic revenue: {revenue}")
print(diff_dplm[-20:]) # print first 20 results

[0.6910947  1.27843893 1.0940666  1.02223777 0.9462198  1.05305249
 0.79061086 1.13342619 1.24225992 1.28518051]
[0.98926579 1.12006768 1.10855944 1.0228186  0.93831595 1.1271953
 0.88544887 1.08839671 0.99702215 1.13879167]
[1.03064564 1.11848375 1.02024087 1.10948981 0.99559667 1.02561255
 0.98869853 1.10638004 1.04024645 1.07308808]
[1.07076896 1.1426658  1.05877735 1.05167663 1.00153519 1.13526058
 1.07660178 1.06423735 1.00509076 1.02125323]
[1.03931965 1.11345259 1.07279337 1.03708435 1.05728996 1.13091132
 1.05154387 1.092419   1.01462012 1.04019477]
[1.05070048 1.10916328 1.07694991 1.04712715 1.0605825  1.11261206
 1.0776834  1.04426491 1.05095298 1.04407662]
[1.04956867 1.09277148 1.10910989 1.07651082 1.04161595 1.08685437
 1.06723273 1.04268705 1.04802063 1.07022546]
SLPM Dynamic revenue: 11030.173656259723
[-244.188874865682, -245.31845732546753, -246.44803978525306, -247.57762224503858, -248.70720470482593, -249.83678716461327, -250.9663696243988, -252.09595208418432, -25

In this question, instead of using a fixed dual price, we update our dual price on k= [50, 100, 200, 400, 800, ...], and using this stratege, we can get better performance than with fixed dual price, 10905.843288804756 compare to revenue in question above.

Also we print all dual price on each update time point. As we can see, it approches to the ground true price p = [1,1,...,1,1] we set before.

In [73]:
# Problem 3
def run_ahdla(bids, k, n, b_i, offline_revenue_value):
    global diff_ahdla
    batch = 50
    revenue = 0
    dual_price = np.zeros(m)
    remaining_capacity = b_i
    target = batch*k

    for i, (a_k, pi_k) in enumerate(bids):
        # Allocate based on the decision rule using y_bar
        if pi_k > np.dot(a_k, dual_price) and all(remaining_capacity - a_k >= 0):
            revenue += pi_k
            remaining_capacity -= a_k
        diff_ahdla.append(revenue-((i+1)/n)*offline_revenue_value)
        if i == target and i < n-1:
            _, dual_price, _ = solve_partial_lp_dual(bids, i+1, n, (n/(n-i-1))*remaining_capacity)
            target += batch

    return revenue

revenue = run_ahdla(bids_fixed, 1, n, b_i, offline_revenue_value)
print(f"Action-history-dependent Learning Algorithm revenue: {revenue}")
print(diff_ahdla[-20:]) # print first 20 results

Action-history-dependent Learning Algorithm revenue: 11236.862071318921
[-49.364971743409114, -50.49455420319464, -51.62413666298016, -52.75371912276569, -53.88330158255303, -55.012884042340374, -56.1424665021259, -52.160221494828875, -53.2898039546144, -54.41938641440174, -55.54896887418727, -56.67855133397279, -57.808133793758316, -52.18503178370338, -53.31461424349072, -54.44419670327625, -55.57377916306177, -56.7033616228473, -57.83294408263464, -58.962526542420164]


In Action-history-dependent Learning Algorithm, instead of updating dual price in a frequency of [50, 100, 200, ...], we update it for each batch, we set 50 here. And as we can see, the revenue is higher than what we got from problem 2.

Hence Action-history-dependent Learning Algorithm have better performance. Becuase it also update with remaining capacity information.

As we can see in the difference vector, compared with problem 2, (see result on problem2), Action-history-dependent Learning Algorithm is closer to partial of optimal solution at each time point.

# Problem 4
## Convexity

The Formulation (3) in question is as follows:

\begin{align*}
\text{minimize}_{\bar{\mathbf{y}}} \; & \;\mathbf{d}^T \bar{\mathbf{y}} + \mathbb{E}\left(\pi - \mathbf{a}^T \bar{\mathbf{y}}\right)^+ \\
\text{s.t. } & \bar{\mathbf{y}} \geq 0
\end{align*}

where $\mathbf{d} = \frac{\mathbf{b}}{n}$ and $(\cdot)^+ = \max \{\cdot, 0\}$.

To identify whether (3) is a convex optimization problem or not, we can analyze each part of (3) separately.

For the first part $\mathbf{d}^T \bar{\mathbf{y}}$ of (3), $\mathbf{d}^T \bar{\mathbf{y}}$ is liner in $\bar{\mathbf{y}}$. We know that linear functions are both convex and concave.

For the second part $\mathbb{E}\left(\pi - \mathbf{a}^T \bar{\mathbf{y}}\right)^+$, since $\pi - \mathbf{a}^T \bar{\mathbf{y}}$ is linear in $\bar{\mathbf{y}}$. What $(\cdot)^+$ does is to get maximum value between this linear function and zero, since both of them are convex, $\left(\pi - \mathbf{a}^T \bar{\mathbf{y}}\right)^+$ is also convex. The last part we need to consider is the expectation. From Jensen's inequality:

$$\mathbb{E}[f(X)]≥f(\mathbb{E}[X])$$

we know that if $f(x)$ is convex, then $\mathbb{E}[f(x)]$ is also convex. Therefore, $\mathbb{E}\left(\pi - \mathbf{a}^T \bar{\mathbf{y}}\right)^+$ is convex.

Since both parts of (3) are convex, we know that (3) is a convex optimization problem.

## Connection

The dual problem of (1) is:

$$
\min \sum_{i=1}^{m} b_iy_i + \sum_{j=1}^{n} \beta_j\\
\text{s.t.} \sum_{i=1}^{m} a_{ij}y_i + \beta_j \geq \pi_j, \quad j=1,\ldots,n.\\
y_i, \beta_j \geq 0 \text{ for all } i, j
$$

From previous questions, we know that the primal optimal solution satisfies:
$$
x_j^* =
\begin{cases}
1, & \text{if } \pi_j > a_j^T y_k^* \\
0, & \text{if } \pi_j \leq a_j^T y_k^*.
\end{cases}
$$

the optimal solution $x_j$ may take non-integer values. That means the optimal solution of primary problem (1) highly depends on $y$. So by plugging the constraints $\sum_{i=1}^{m} a_{ij}y_i + \beta_j \geq \pi_j$ into the objective function, an equivalent form of the dual problem can be obtained as:
$$
\min \sum_{i=1}^{m} b_i y_i + \sum_{j=1}^{n} \left( \pi_j - \sum_{i=1}^{m} a_{ij} y_i \right)^+\\
\text{s.t. } y_i \geq 0, \quad i = 1, \ldots, m.
$$

And if we devide it by $n$, from above we know that $\mathbf{d} = \frac{\mathbf{b}}{n}$, so we now have:
$$
\min f_n(\bar{\mathbf{y}}) = \sum_{i=1}^{m} d_i y_i + \frac{1}{n} \sum_{j=1}^{n} \left( \pi_j - \sum_{i=1}^{m} a_{ij} y_i \right)^+\\
\text{s.t. } y_i \geq 0, \quad i = 1, \ldots, m.
$$

Since $(\pi, a)$ is a sequence of i.i.d. random vectors, we can also write this formulation as:

$$
\begin{align*}
\text{minimize}_{\bar{\mathbf{y}}} \; & \;\mathbf{d}^T \bar{\mathbf{y}} + \mathbb{E}\left(\pi - \mathbf{a}^T \bar{\mathbf{y}}\right)^+ \\
\text{s.t. } & \bar{\mathbf{y}} \geq 0
\end{align*}
$$

where the expectation is taken with respect to $(\pi, a)$.
